# Predictive Analytics: Milestone 3
#### Joshua Greenert, Gabriel Avinaz, and Mithil Patel
#### DSC630-T301 Predictive Analytics
#### 12/26/2022

In [32]:
# Import the required libaries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [2]:
# Pull in the data.  Each person can have their own path.  Comment out the one that isn't yours if different.
# Josh G
df_steam_reviews = pd.read_csv('../../../../../Downloads/steam_reviews.csv', low_memory=False)
# Gabe A
# df_steam_reviews = pd.read_csv('steam_reviews.csv', low_memory=False)

In [3]:
# Show the head of the dataframe to confirm the data is present.
df_steam_reviews.head(5)

,Unnamed: 0,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1.611343e+09
1,1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1.611386e+09
2,2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1.611384e+09
3,3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
4,4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1.610788e+09


In [24]:
df_steam_games = pd.read_csv("steam_games.csv", sep=';', engine='python')

In [25]:
df_steam_games.sample(10)

,App ID,Name,Short Description,Developer,Publisher,Genre,Tags,Type,Categories,Owners,...,Price,Initial Price,Discount,CCU,Languages,Platforms,Release Date,Required Age,Website,Header Image
39191,548130,Scribble Ships,Scribble Ships is a 2D top-down arcade shooter...,Slothzilla Games,(none),Action,"Action: 24, Arcade: 13, Indie: 12",game,"Single-player, Steam Achievements, Steam Leade...","0 .. 20,000",...,199,199,0,0,English,"windows, mac, linux",2017/03/29,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
33331,383190,SUMICO - The Numbers Game,Sumico is a very original number puzzle game. ...,Ludomotion,Ludomotion,"Casual, Indie","Indie: 22, Casual: 20, Puzzle: 6",game,Single-player,"0 .. 20,000",...,499,499,0,0,"English, French, Italian, German, Spanish - Sp...","windows, mac, linux",2015/07/3,0,http://www.playsumico.com,https://cdn.akamai.steamstatic.com/steam/apps/...
23876,1868510,Wordish,Wordish takes word games to the next level! Fi...,Dragon Slumber,Dragon Slumber,"Casual, Indie","Casual: 61, Word Game: 48, Puzzle: 38, Singlep...",game,"Multi-player, Single-player, Steam Cloud, Onli...","0 .. 20,000",...,399,399,0,0,English,"windows, mac, linux",2022/01/25,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
2767,1078340,Mongrel,The only roguelite where your only form of pub...,Fischmell,Fischmell,"Action, Adventure, Casual, Indie, Early Access","Early Access: 149, Roguevania: 135, Dark Humor...",game,"Single-player, Captions available","0 .. 20,000",...,1299,1299,0,0,English,windows,2022/02/19,0,https://mongrel-game.com/,https://cdn.akamai.steamstatic.com/steam/apps/...
27776,2144910,Blueprints,A parkour platformer where the more levels you...,"Conabe Games, MADD Systems",Conabe,"Action, Casual, Indie","Casual: 294, Platformer: 268, 3D Platformer: 2...",game,"Single-player, Steam Achievements, Steam Cloud...","0 .. 20,000",...,499,499,0,0,English,windows,2022/10/4,0,https://www.blueprintsgame.com,https://cdn.akamai.steamstatic.com/steam/apps/...
8335,1246870,Harvest Moon: One World,"Trek across beaches, the desert, and even a vo...","Natsume Inc., Appci",Natsume Inc.,"Adventure, RPG, Simulation","Adventure: 72, Simulation: 64, RPG: 62, Farmin...",game,"Single-player, Partial Controller Support, Ste...","0 .. 20,000",...,4999,4999,0,7,"English, French, German, Spanish - Spain, Simp...",windows,2021/09/7,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
50458,851320,SlimeGear,"Help Pinion and Cog, two Slime Gears, run thro...",DitheryGames,DitheryGames,"Adventure, Casual, Indie","Casual: 21, Indie: 21, Adventure: 20",game,Single-player,"0 .. 20,000",...,99,99,0,0,English,windows,2018/08/6,0,NaN,https://cdn.akamai.steamstatic.com/steam/apps/...
8378,1248360,WalkinVR,WalkinVR Driver is a software program for peop...,2MW,2MW,Utilities,"Utilities: 22, VR: 13, Controller: 11",game,NaN,"20,000 .. 50,000",...,0,0,0,8,English,windows,2020/06/29,0,https://www.walkinvrdriver.com,https://cdn.akamai.steamstatic.com/steam/apps/...
32275,357330,Space Beast Terror Fright,S.B.T.F. is a very hard arcade-style / rogue-l...,nornware AB,nornware AB,"Action, Indie","Action Roguelike: 51, Action: 44, Horror: 38, ...",game,"Multi-player, Single-player, Co-op, Partial Co...","100,000 .. 200,000",...,1499,1499,0,18,English,windows,2022/06/3,0,http://www.nornware.com/,https://cdn.akamai.steamstatic.com/steam/apps/...
23791,1863570,YUZA,YUZA is a game about jumping and coins!,Dnovel,Trinity Project,"Action, Adventure, Indie","Adventure: 68, Action: 62, Arcade: 39, 2D Plat...",game,Steam Achievements,"0 .. 20,000",...,99,99,0,0,"English, Russian",windows,2022/03/6,0,https://www.facebook.com/DnovelGames/,https://cdn.akamai.steamstatic.com/steam/apps/...


### Data Preparation

In [4]:
# Using the language column, we can remove all other languages besides english.
df_steam_reviews = df_steam_reviews[df_steam_reviews['language'] == 'english']

In [5]:
# Drop the columns that we don't need.
df_steam_reviews = df_steam_reviews.drop(['Unnamed: 0', 'review_id', 'language','author.num_games_owned', 'author.last_played' ], axis = 1)

In [6]:
# Make dummies of the columns that can conform.
df_reviews_dummies = pd.get_dummies(df_steam_reviews, columns=['recommended', 'steam_purchase', 'received_for_free',
                                                              'written_during_early_access'])

In [7]:
df_reviews_dummies.head(5)

,app_id,app_name,review,timestamp_created,timestamp_updated,votes_helpful,votes_funny,weighted_vote_score,comment_count,author.steamid,...,author.playtime_last_two_weeks,author.playtime_at_review,recommended_False,recommended_True,steam_purchase_False,steam_purchase_True,received_for_free_False,received_for_free_True,written_during_early_access_False,written_during_early_access_True
3,292030,The Witcher 3: Wild Hunt,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,0,0,0.0,0,76561199054755373,...,3200.0,5524.0,0,1,0,1,1,0,1,0
5,292030,The Witcher 3: Wild Hunt,"good story, good graphics. lots to do.",1611379264,1611379264,0,0,0.0,0,76561198170193529,...,823.0,823.0,0,1,0,1,1,0,1,0
6,292030,The Witcher 3: Wild Hunt,"dis gud,",1611379091,1611379091,0,0,0.0,0,76561198119302812,...,3398.0,4192.0,0,1,0,1,1,0,1,0
18,292030,The Witcher 3: Wild Hunt,favorite game of all time cant wait for the Ne...,1611373086,1611373086,0,0,0.0,0,76561198065591528,...,177.0,23329.0,0,1,0,1,1,0,1,0
20,292030,The Witcher 3: Wild Hunt,Why wouldn't you get this,1611371978,1611371978,0,0,0.0,0,76561198996835044,...,2004.0,8557.0,0,1,0,1,1,0,1,0


In [37]:
# Drop the columns that we don't need.
df_steam_games = df_steam_games[['App ID', 'Name', 'Short Description', 'Positive Reviews', 'Negative Reviews']]
df_steam_games['Total Reviews'] = df_steam_games['Positive Reviews'] + df_steam_games['Negative Reviews']
df_steam_games.sample(10)

,App ID,Name,Short Description,Positive Reviews,Negative Reviews,Total Reviews
35848,451780,Trillion: God of Destruction,An ancient god consumes your world. Now you mu...,299,144,443
796,1022040,Candy Mandy,Classic game in the popular &quot;three in a r...,2,3,5
25007,1931890,Trivia Party,A mix of trivia and a classic board game. What...,6,1,7
42759,652600,Descent - Silence of Mind,"Hypnosis, memories, delusions and a disturbing...",56,30,86
49043,812800,Viki Spotter: School,Here is a puzzle game that will put you in fro...,61,5,66
40521,583860,PhotoZoom Classic 6,Take photos from ordinary to extraordinary wit...,6,0,6
2628,1073720,Navigation Bar,&quot;Navigation Bar&quot; can greatly save th...,12,8,20
21575,1750290,Ninja: Rise of a Hero,The path of the ninja is hard! Can you save yo...,1,0,1
5313,1151940,Super Minesweeper attACK,"You can play minesweeper, but you can have cus...",5,1,6
13020,1405040,GoVenture Entrepreneur,The most realistic small business startup and ...,2,1,3


In [33]:
# Create the lemmatizer, porter, and codes.
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'}

# Define a function that processes the lemmatizing on each sentence.
def process_sentence_lemmatize(text):
    # changing to lower case, tokenizing, and lemmatizing each of the descriptions
    text = str(text).lower()
    temp_sent =[]
    words = word_tokenize(text)
    tags = nltk.pos_tag(words)
    
    # For loop to remove verb portion of words
    for i, word in enumerate(words):
        if tags[i][1] in VERB_CODES: 
            lemmatized = lemmatizer.lemmatize(word, 'v')
        else:
            lemmatized = lemmatizer.lemmatize(word)
        if lemmatized not in stop_words and lemmatized.isalpha():
            temp_sent.append(lemmatized)
            
    # joining tokenized words and removing contractions
    finalsent = ' '.join(temp_sent)
    finalsent = finalsent.replace("n't", " not")
    finalsent = finalsent.replace("'m", " am")
    finalsent = finalsent.replace("'s", " is")
    finalsent = finalsent.replace("'re", " are")
    finalsent = finalsent.replace("'ll", " will")
    finalsent = finalsent.replace("'ve", " have")
    finalsent = finalsent.replace("'d", " would")
    
    return finalsent

# Define a function that performs additional preparation by replacing contractions with full words.
def process_sentence_stemm(text):
    # changing to lower case, tokenizing, and stemm each of the descriptions
    text = str(text).lower()
    temp_sent =[]
    words = word_tokenize(text)
    tags = nltk.pos_tag(words)
    for word in words:
        words = porter.stem(word) 
    words = ' '.join([word for word in words])

    # joining tokenized words and removing contractions
    finalsent = ' '.join(temp_sent)
    finalsent = finalsent.replace("n't", " not")
    finalsent = finalsent.replace("'m", " am")
    finalsent = finalsent.replace("'s", " is")
    finalsent = finalsent.replace("'re", " are")
    finalsent = finalsent.replace("'ll", " will")
    finalsent = finalsent.replace("'ve", " have")
    finalsent = finalsent.replace("'d", " would")
    return finalsent

In [34]:
# Download the required libraries.
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Gabe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## WARNING: This lemmatization takes up to 6 hours to complete.

In [11]:
# Calling the process_sentence_lemmatize function through the apply method.
df_reviews_dummies['prepped_review_lemm'] = df_reviews_dummies.review.apply(process_sentence_lemmatize)
df_reviews_dummies.sample(10)

,app_id,app_name,review,timestamp_created,timestamp_updated,votes_helpful,votes_funny,weighted_vote_score,comment_count,author.steamid,...,author.playtime_at_review,recommended_False,recommended_True,steam_purchase_False,steam_purchase_True,received_for_free_False,received_for_free_True,written_during_early_access_False,written_during_early_access_True,prepped_review_lemm
11662534,239140,Dying Light,good game can i get my badge now plz,1561837608,1561837608,0,1,0.000000,0,76561198125978430,...,6162.0,0,1,1,0,1,0,1,0,good game get badge plz
8734990,508440,Totally Accurate Battle Simulator,"It`s a great game with lots of units, wacky ph...",1591409179,1591409179,0,0,0.000000,0,76561197980810778,...,6825.0,0,1,0,1,1,0,0,1,great game lot unit wacky physic cool map watc...
1553901,4000,Garry's Mod,don't do it,1358385102,1403953474,0,0,0.477454,0,76561198046475532,...,136497.0,1,0,0,1,1,0,1,0,
11723714,239140,Dying Light,"I slept on this game for a whole year, and i'm...",1461730763,1461730763,0,0,0.000000,0,76561198046541591,...,620.0,0,1,1,0,1,0,1,0,sleep game whole year glad come sens buy defin...
5424579,782330,DOOM Eternal,superior to the previous one in every possible...,1586116993,1586116993,3,0,0.508857,0,76561197978654658,...,3694.0,0,1,0,1,1,0,1,0,superior previous one every possible way
196239,292030,The Witcher 3: Wild Hunt,The game works perfect with Proton and creates...,1577227250,1577227250,0,0,0.000000,0,76561197993576059,...,2060.0,0,1,0,1,1,0,1,0,game work perfect proton create really nice at...
15589221,582010,Monster Hunter: World,"Amazing game, if you find it on sale do yourse...",1562009601,1562009601,0,0,0.000000,0,76561198074453016,...,3651.0,0,1,0,1,1,0,1,0,amazing game find sale favor buy upcoming upda...
17674808,626690,Sword Art Online: Fatal Bullet,Customizations are so realistic I m fappin jus...,1519413690,1519413690,1,1,0.521739,0,76561198263533863,...,366.0,0,1,0,1,0,1,1,0,customizations realistic fappin jus lookin cha...
3216993,285190,"Warhammer 40,000: Dawn of War III",Very fun game! Clearly a lot of care and thoug...,1500864135,1500864135,8,1,0.493968,0,76561198056106362,...,1445.0,0,1,1,0,1,0,1,0,fun game clearly lot care think go game clear ...
7812166,252490,Rust,"Before I begin, Rust is not a ""Zombie"" Surviva...",1391806636,1391806636,1,0,0.521739,0,76561198120541302,...,3103.0,0,1,0,1,1,0,0,1,begin rust zombie survival game survival game ...


In [38]:
# Calling the process_sentence_lemmatize function through the apply method.
df_steam_games['prepped_description_lemm'] = df_steam_games['Short Description'].apply(process_sentence_lemmatize)
df_steam_games.sample(10)

,App ID,Name,Short Description,Positive Reviews,Negative Reviews,Total Reviews,prepped_description_lemm
11371,13520,Far Cry,A tropical paradise seethes with hidden evil i...,5845,1251,7096,tropical paradise seethes hidden evil far cunn...
18391,1597820,Pazing By,Pazing By is a GRAZE-oriented with 2D platform...,8,0,8,pazing platformer bullet hell game player need...
17526,1565630,Doge to the Moon,Doge to the Moon is an endless runner game bas...,4,0,4,doge moon endless runner game base wsb dogecoi...
13329,1415410,Farm Manager 2021: Prologue,Get ready for a logistic challenge in the new ...,703,204,907,get ready logistic challenge new even better f...
43379,667510,The Elmian Warrior,Get yourself ready to come across ancient ruin...,2,2,4,get ready come across ancient ruin abandon cit...
12323,1382890,Mystic Midway: Rest in Pieces,Mystic Midway: Rest in Pieces – Somewhere betw...,5,0,5,mystic midway rest piece somewhere heaven eart...
4915,1141150,The Defender: Farm and Castle 2,You are a magician who will protect the castle...,7,5,12,magician protect castle invasion monster manag...
47143,763760,My Coloring Book: Food and Beverage,Presenting you a simple yet inspiring coloring...,90,19,109,present simple yet inspire color book help spe...
37883,511490,Malavision: The Beginning,A politician runs away upon sued corruption ac...,32,17,49,politician run away upon sue corruption act pr...
22779,1811140,SEARCH ALL - STEAMPUNK,is a game in the genre of &quot;hidden object&...,28,1,29,game genre quot hide object quot need find bol...


## Checkpoint File:  File present to ensure you don't have to rerun the 6 hour code above.

In [ ]:
df_reviews_dummies.to_csv('Prepped_test_out.csv', index=False)

In [39]:
df_steam_games.to_csv('Prepped_games_out.csv', index=False)

----------------------------------------------------------------------

## New Start Point: Model Finalization

In [1]:
# Import the required libaries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [56]:
# Pull in the dataframe.

# Josh G
# df_reviews_dummies = pd.read_csv('../../../../../Downloads/Prepped_test_out.csv', low_memory=False)
# df_reviews = pd.read_csv('../../../../../Downloads/steam_reviews.csv', low_memory=False)
# Gabe
df_reviews_dummies = pd.read_csv('Prepped_test_out.csv', low_memory=False)
df_reviews = pd.read_csv('steam_reviews.csv', low_memory=False)
df_steam_games = pd.read_csv('Prepped_games_out.csv')
df_steam_games = df_steam_games.set_index('App ID')

In [3]:
# merge the two columns into one
df_reviews_dummies["recommended"] = df_reviews_dummies['recommended_False'] + df_reviews_dummies['recommended_True']

# Set the user recommendation dataframes.
user_rec_df_reviews = df_reviews[["app_id","author.steamid", "recommended"]].copy()
user_rec_df_prepped = df_reviews_dummies[["app_id","author.steamid", "recommended"]].copy()

In [4]:
# Set the values that will be used for the matrix dimensions.
n_ratings_reviews = len(user_rec_df_reviews)
n_games_reviews = len(user_rec_df_reviews['app_id'].unique())
n_users_reviews = len(user_rec_df_reviews['author.steamid'].unique())

n_ratings_prepped = len(user_rec_df_prepped)
n_games_prepped = len(user_rec_df_prepped['app_id'].unique())
n_users_prepped = len(user_rec_df_prepped['author.steamid'].unique())

# Print the results to confirm they have values.
print(n_ratings_reviews, n_games_reviews, n_users_reviews)
print(n_ratings_prepped, n_games_prepped, n_users_prepped)

21747371 315 12406560
9635437 315 5287718


In [5]:
# maps idices to users and game IDs
user_map_reviews = dict(zip(np.unique(user_rec_df_reviews["author.steamid"]), list(range(n_users_reviews))))
game_map_reviews = dict(zip(np.unique(user_rec_df_reviews["app_id"]), list(range(n_games_reviews))))

# Confirm that the total reviews equal the n_games_reviews. Should be true, true.
print(len(user_map_reviews) == n_users_reviews)
print(len(game_map_reviews) == n_games_reviews)

user_i_map_reviews = dict(zip(list(range(n_users_reviews)), np.unique(user_rec_df_reviews["author.steamid"])))
game_i_map_reviews = dict(zip(list(range(n_games_reviews)), np.unique(user_rec_df_reviews["app_id"])))

# Perform the same operations on the prepped items.
user_map_prepped = dict(zip(np.unique(user_rec_df_prepped["author.steamid"]), list(range(n_users_prepped))))
game_map_prepped = dict(zip(np.unique(user_rec_df_prepped["app_id"]), list(range(n_games_prepped))))

# Confirm that the total reviews equal the n_games_reviews. Should be true, true.
print(len(user_map_prepped) == n_users_prepped)
print(len(game_map_prepped) == n_games_prepped)

user_i_map_prepped = dict(zip(list(range(n_users_prepped)), np.unique(user_rec_df_prepped["author.steamid"])))
game_i_map_prepped = dict(zip(list(range(n_games_prepped)), np.unique(user_rec_df_prepped["app_id"])))

True
True
True
True


In [6]:
# creating indices for csr_matrix
user_index_reviews = [user_map_reviews[i] for i in user_rec_df_reviews['author.steamid']]
game_index_reviews = [game_map_reviews[i] for i in user_rec_df_reviews['app_id']]

user_index_prepped = user_rec_df_prepped['author.steamid'].map(user_map_prepped, na_action='ignore').fillna(-1).astype(int)
game_index_prepped = [game_map_prepped[i] for i in user_rec_df_prepped['app_id']]

In [7]:
# creates csr matrixes
matrix_reviews = csr_matrix((user_rec_df_reviews["recommended"], (game_index_reviews, user_index_reviews)), shape=(n_games_reviews, n_users_reviews))
matrix_prepped = csr_matrix((user_rec_df_prepped["recommended"], (game_index_prepped, user_index_prepped)), shape=(n_games_prepped, n_users_prepped))

In [8]:
# creating a dictionary to pull the name of games from based on IDs
game_names_reviews = dict(zip(df_reviews['app_id'], df_reviews['app_name']))
game_names_prepped = dict(zip(df_reviews_dummies['app_id'], df_reviews_dummies['app_name']))

In [21]:
'''
This function uses both datasets to determine the best match for returning a list of games.  It uses KNN
along with the cosine function to find similiaries and store them into a list of neighbor ids.  These ids
are then compared for accuracy and the highest matches are returned.

@param game_id: the id that references the game found with Steam.
@param total_matches: the total number of matches to return
'''
def find_related_games(game_id, total_matches, usePrepped = True):
    
    # Set variables
    reviews_neighbour_ids_with_distance = {}
    if usePrepped:
        prepped_neighbour_ids_with_distance = {}
    
    # Prepare index/vectorizations.
    game_index_reviews = game_map_reviews[game_id]
    game_vector_reviews = matrix_reviews[game_index_reviews]
    
    
    if usePrepped:
        game_index_prepped = game_map_prepped[game_id]
        game_vector_prepped = matrix_prepped[game_index_prepped]
    
    # Increment total matches.
    total_matches += 1
    
    # Set the KNN model and fit them.
    kNN_reviews = NearestNeighbors(algorithm = 'brute', metric='cosine')
    kNN_reviews.fit(matrix_reviews)
    
    if usePrepped:
        kNN_prepped = NearestNeighbors(algorithm = 'brute', metric='cosine')
        kNN_prepped.fit(matrix_prepped)
    
    # reshape and determine distances for KNN values.
    game_vec_reshaped_reviews = game_vector_reviews.reshape(1,-1)
    distances_reviews, indices_reviews = kNN_reviews.kneighbors(game_vec_reshaped_reviews, n_neighbors=total_matches)
    
    if usePrepped:
        game_vec_reshaped_prepped = game_vector_prepped.reshape(1,-1)
        distances_prepped, indices_prepped = kNN_prepped.kneighbors(game_vec_reshaped_prepped, n_neighbors=total_matches)
    
    # Loop through and flatten the distances provided
    for i in range(0,len(distances_reviews.flatten())):
        n = indices_reviews.flatten()[i]
        neighbour_id = game_i_map_reviews[n]
        reviews_neighbour_ids_with_distance[game_names_reviews[neighbour_id]] = distances_reviews.flatten()[i]
    reviews_neighbour_ids_with_distance.pop(game_names_reviews[game_id], None) # removes the same game
    
    if usePrepped:
        for i in range(0,len(distances_prepped.flatten())):
            n = indices_prepped.flatten()[i]
            neighbour_id = game_i_map_prepped[n]
            prepped_neighbour_ids_with_distance[game_names_prepped[neighbour_id]] = distances_prepped.flatten()[i]
        prepped_neighbour_ids_with_distance.pop(game_names_prepped[game_id], None) 
    
    # Sort the data by accuracy
    sorted_neighbour_ids_with_distance_reviews = sorted(reviews_neighbour_ids_with_distance.items(), key=lambda x: x[1], reverse=True)
    if usePrepped:
        sorted_neighbour_ids_with_distance_prepped = sorted(prepped_neighbour_ids_with_distance.items(), key=lambda x: x[1], reverse=True)
    else:
        sorted_neighbour_ids_with_distance_prepped = []
        
    # finalize the list amongst the two created.
    combined_list = sorted_neighbour_ids_with_distance_reviews + sorted_neighbour_ids_with_distance_prepped
    sorted_combined_list = sorted(combined_list, key=lambda x: x[1], reverse=True)
    
    # Print the games and their related accuracy.
    count = 0
    
    print(f"Games related to: {game_names_reviews[game_id]}\n")
    for game_name, accuracy in sorted_combined_list:
        if count == total_matches:
            break
        else:
            print(f"{game_name}: {accuracy:.2f}")
            # count += 1

In [19]:
# Set a test game id and run the function.
game_id = 292030
total_matches = 5
 
find_related_games(game_id, total_matches)

Games related to: The Witcher 3: Wild Hunt

Rise of the Tomb Raider: 0.93
Fallout 4: 0.93
The Elder Scrolls V: Skyrim: 0.93
Grand Theft Auto V: 0.93
DARK SOULS™ III: 0.93
Dying Light: 0.93


In [22]:
# Set a test game id and run the function.
game_id = 582010
total_matches = 5
 
find_related_games(game_id, total_matches)

Games related to: Monster Hunter: World

Divinity: Original Sin 2: 0.96
Divinity: Original Sin 2: 0.95
NieR:Automata™: 0.95
The Witcher 3: Wild Hunt: 0.95
The Witcher 3: Wild Hunt: 0.95
NieR:Automata™: 0.95
Sekiro™: Shadows Die Twice: 0.94
Sekiro™: Shadows Die Twice: 0.93
DARK SOULS™ III: 0.93
DARK SOULS™ III: 0.93


In [42]:
# Create the td-idf vectorizer. 
tfidfvec = TfidfVectorizer()

tfidf_descriptions = tfidfvec.fit_transform((df_steam_games['prepped_description_lemm'].astype('U')))

# Perform a test using a fixed term.
search = "Tom Clany's Rainbow Six Siege"
search_processed = re.sub("[^a-zA-Z0-9 ]","",search.lower())

vec= tfidfvec.transform([search_processed])

# Add cosine similarity
similarity = cosine_similarity(tfidf_descriptions)
indices = np.argpartition(similarity, -10)[-10:]

# Set the results.
results = df_steam_games.iloc[indices]

# Print the resulting dataframe.
results[::-1]

In [60]:
tfidfvec = TfidfVectorizer()

tfidf_gamedesc = tfidfvec.fit_transform((df_steam_games['prepped_description_lemm'].values.astype('U')))

cos_sim = cosine_similarity(tfidf_gamedesc, tfidf_gamedesc)

# functions for retrieving similar game IDs from cos_sim
indices = pd.Series(df_steam_games.index)
  

In [95]:
df_steam_games.index

Int64Index([     10, 1000000, 1000010, 1000030, 1000040, 1000080, 1000100,
            1000110, 1000130, 1000280,
            ...
             999750,  999760,  999830,  999840,  999860,  999880,  999890,
             999900,  999930,  999990],
           dtype='int64', name='App ID', length=55691)

In [119]:
def get_related_games_desc(id):
    recommended_games =  pd.DataFrame()
    index = indices[indices == id].index[0]
    similarity_scores = pd.Series(cos_sim[index]).sort_values(ascending = False)
    top = list(similarity_scores.iloc[1:11].index)
    recommended_games['Score'] = similarity_scores.iloc[1:11]
    app_ids = []
    for i in top:
        app_ids.append(list(df_steam_games.index)[i])
    recommended_games['App ID'] = app_ids
    return recommended_games

In [120]:
game_id = 292030
total_matches = 5

recommended_games = get_related_games_desc(game_id)
print(f"Games related to: {df_steam_games.loc[df_steam_games.index == game_id].Name.values[0]}")
for i, val in enumerate(recommended_games["App ID"]):
    print(f"{df_steam_games.loc[df_steam_games.index == val].Name.values[0]}: {recommended_games.Score.values[i]:.5f}")

Games related to: The Witcher 3: Wild Hunt
Rage Wars: 0.26487
Salvation Prophecy: 0.24443
Fated Blade: 0.23047
Straight Ahead: 0.22349
The Witcher 3: Wild Hunt Game of the Year Edition: 0.21558
Heroes of Myth: 0.21332
Neocense: 0.18114
Emily: Displaced: 0.17830
Arena Hero: 0.17806
Lords of Kingdoms: 0.17480


In [122]:
game_id = 248450
total_matches = 5

recommended_games = get_related_games_desc(game_id)
  
print(f"Games related to: {df_steam_games.loc[df_steam_games.index == game_id].Name.values[0]}")
for i in recommended_games["App ID"]:
    print(df_steam_games.loc[df_steam_games.index == i].Name.values[0])

Games related to: Salvation Prophecy
Fated Blade
Heroes of Myth
Galactic Battles
The Witcher 3: Wild Hunt
Revelations 2012
Galaxy Annihilation
Chimeras: The Signs of Prophecy Collector's Edition
DragonScales 3: Eternal Prophecy of Darkness
ViSP - Virtual Space Port
Coeus Plan


'Divide and Conquer: The Board Game'